# Day 13 - Distress Signal


In [17]:
testData = """[1,1,3,1,1]
[1,1,5,1,1]

[[1],[2,3,4]]
[[1],4]

[9]
[[8,7,6]]

[[4,4],4,4]
[[4,4],4,4,4]

[7,7,7,7]
[7,7,7]

[]
[3]

[[[]]]
[[]]

[1,[2,[3,[4,[5,6,7]]]],8,9]
[1,[2,[3,[4,[5,6,0]]]],8,9]"""



In [ ]:
#let's test out some eval() 
x = '[1,2,3]'
y = eval(x)
print(y)
print(y[1])


## Comparison rules
Packet data consists of lists and integers. Each list starts with [, ends with ], and contains zero or more comma-separated values (either integers or other lists). Each packet is always a list and appears on its own line.

When comparing two values, the first value is called left and the second value is called right. Then:

- If both values are integers, the lower integer should come first. If the left integer is lower than the right integer, the inputs are in the right order. If the left integer is higher than the right integer, the inputs are not in the right order. Otherwise, the inputs are the same integer; continue checking the next part of the input.
- If both values are lists, compare the first value of each list, then the second value, and so on. If the left list runs out of items first, the inputs are in the right order. If the right list runs out of items first, the inputs are not in the right order. If the lists are the same length and no comparison makes a decision about the order, continue checking the next part of the input.
- If exactly one value is an integer, convert the integer to a list which contains that integer as its only value, then retry the comparison. For example, if comparing [0,0,0] and 2, convert the right value to [2] (a list containing 2); the result is then found by instead comparing [0,0,0] and [2].


Using these rules, you can determine which of the pairs in the example are in the right order:

## Approach
Rather than parse the strings manually, just going to EVAL() the crap out of this one!

Comparing two packets requires a level of recursion... let's just get on and sling some code and see what drops out.

Need a three way boolean: True, False, unknown.
Can None stand in? Probably.

In [ ]:
def process(input:str):
    #process the input string
    rows = input.splitlines()
    pairCount = 0
    correctlyOrderPairs = []
    for i in range(0,len(rows),3):
        pairCount += 1
        print('== Pair '+str(pairCount)+' ==')
        left = eval(rows[i])
        right = eval(rows[i+1])
        if compare(left,right):
            print ('Left side is smaller, so inputs are in the right order')
            correctlyOrderPairs.append(pairCount)
        else:
            print('Right side is smaller, so inputs are not in the right order')
    #calculate sum
    print('---')
    print('Correctly ordered pairs: '+str(correctlyOrderPairs))
    print('Sum of indicies of correctly ordered pairs: '+str(sum(correctlyOrderPairs)))

def compare(left, right)->bool:
    #compare left and right side. Return true is in order, false is not.
    print('Compare '+str(left)+' vs '+str(right))
    #If both values are integers, the lower integer should come first
    if isinstance(left, int) and isinstance(right, int):
        if left < right:
            return True
        elif right < left:
            return False  
        else:
            return None 
    #If both values are lists, compare the first value of each list, then the second value, and so on.
    elif isinstance(left,list) and isinstance(right, list):
        #If the lists are the same length and no comparison makes a decision about the order, continue checking the next part of the input.
        if len(left)==0 and len(right)==0:
            return None
        #If the left list runs out of items first, the inputs are in the right order
        elif len(left)==0:
            return True
        #If the right list runs out of items first, the inputs are not in the right order.
        elif len(right)==0:
            return False
        else:
            for i in range(len(left)):
                result = compare(left[i],right[i])
                if result != None:
                    return result
                #need to check for running out of items... 
                elif i+1==len(left) and i+1==len(right):
                    return None
                elif i+1==len(right):
                    #running out of items on right
                    return False
                elif i+1==len(left) and i+1<len(right):
                    return True


    #If exactly one value is an integer, convert the integer to a list which contains that integer as its only value, then retry the comparison.
    elif isinstance(left, int) and isinstance(right, list):
        return compare([left],right)
    elif isinstance(left,list) and isinstance(right,int):
        return compare(left,[right])
    else:
        print('failed to compare')




#tests
process(testData)

In [ ]:
#using real data
realData = open("day13input.txt").read()
process(realData)


# Part 2: Put them all in order

So, some languages offer the ability to pass in a custom comparitor to an existing sort alogirthm. I hope that's the case here, otherwise I'm googling quicksort!

So... kind of. In Python2, yes, sort took a comparions function. In Python3 it only takes a key function. Apparently functools.cmp_to_key(func) can work magic (bit of research: it does a lot of comparisons, but will only do each pair-wise comparison once).

Also, turns out cmp functions use -1,0,1 as return values. Should have used that in part 1! 

In [25]:
import functools

def compare2(left, right)->int:
    #compare left and right side.
    #If both values are integers, the lower integer should come first
    if isinstance(left, int) and isinstance(right, int):
        if left < right:
            return 1
        elif right < left:
            return -1  
        else:
            return 0 
    #If both values are lists, compare the first value of each list, then the second value, and so on.
    elif isinstance(left,list) and isinstance(right, list):
        #If the lists are the same length and no comparison makes a decision about the order, continue checking the next part of the input.
        if len(left)==0 and len(right)==0:
            return 0
        #If the left list runs out of items first, the inputs are in the right order
        elif len(left)==0:
            return 1
        #If the right list runs out of items first, the inputs are not in the right order.
        elif len(right)==0:
            return -1
        else:
            for i in range(len(left)):
                result = compare2(left[i],right[i])
                if result != 0:
                    return result
                #need to check for running out of items... 
                elif i+1==len(left) and i+1==len(right):
                    return 0
                elif i+1==len(right):
                    #running out of items on right
                    return -1
                elif i+1==len(left) and i+1<len(right):
                    return 1


    #If exactly one value is an integer, convert the integer to a list which contains that integer as its only value, then retry the comparison.
    elif isinstance(left, int) and isinstance(right, list):
        return compare2([left],right)
    elif isinstance(left,list) and isinstance(right,int):
        return compare2(left,[right])
    else:
        raise Exception('failed to compare')

def process2(input:str):
    #process the input string
    inputRows = []
    for i in input.splitlines():
        if i != '':
            inputRows.append(eval(i))

    #add in our divider packets
    div1 = [[2]]
    div2 = [[6]]
    inputRows.append(div1)
    inputRows.append(div2)
    rows = sorted(inputRows, key=functools.cmp_to_key(compare2), reverse=True)

    for r in rows:
        print(r)

    #calculate decoder key - multiple together indicies (starting at 1) of div1 and div2
    div1index = rows.index(div1) + 1
    div2index = rows.index(div2) + 1
    decoderKey = div1index * div2index
    print('---')
    print('Divide 1 index: '+ str(div1index))
    print('Divide 2 index; '+ str(div2index))
    print('Decoder key '+str(decoderKey))


#tests
process2(testData)


[]
[[]]
[[[]]]
[1, 1, 3, 1, 1]
[1, 1, 5, 1, 1]
[[1], [2, 3, 4]]
[1, [2, [3, [4, [5, 6, 0]]]], 8, 9]
[1, [2, [3, [4, [5, 6, 7]]]], 8, 9]
[[1], 4]
[[2]]
[3]
[[4, 4], 4, 4]
[[4, 4], 4, 4, 4]
[[6]]
[7, 7, 7]
[7, 7, 7, 7]
[[8, 7, 6]]
[9]
---
Divide 1 index: 10
Divide 2 index; 14
Decoder key 140


In [26]:
#using real data
realData = open("day13input.txt").read()
process2(realData)


[]
[[]]
[[], []]
[[], [], [1]]
[[], [], [[[5], [9], 5, 7], 5, []], [[3, [1, 2, 6, 3], 9, [3, 2, 7], 0]], [[[5, 9], 5, [8, 1, 7]], [1, 1, [7, 7, 8, 10]]]]
[[], [], [7, 2], [[]]]
[[], [], [[8], [[], 5, [1, 8, 7], 9, 1], [[7, 1, 2, 4, 3], 7, [5, 7, 1, 6, 6], 8, 2], 10], [4, [6, [8, 8, 6, 7]], 1, [[], [5, 10, 3, 2, 7]]], [1, [], [1, [0], 7, [9, 0]]]]
[[], [], [[[9, 3, 4, 2], [4, 5], 1, [8, 0, 7, 8, 4]], 6]]
[[], [[], [3, 5, 0, 7], [1], 2, 10], [3, [], 0, [[9, 0], [0, 0, 2, 5, 9], [1, 6, 2, 6]]]]
[[], [[], [5], [[]], 3, 6]]
[[], [[], 7]]
[[], [[], 7, [], 9, [[0, 9, 4], [8, 0]]]]
[[], [[], 8, 5], [4, 9, [[8, 4, 7, 6, 9], [4]], 3, [[0, 3, 4, 3, 1]]], [3, 5, [[0, 6, 4], 5, [1, 5, 6], 6, [8, 7, 1, 7]]], [1]]
[[], [[[], [3, 4, 0], 9, [], 1], 0, 2, [0, [5, 2, 6, 8], [9, 4, 8, 8]]], [[[10, 6], [5, 6, 4, 3], [], 5, [4]]], [0], []]
[[], [[[], 4, [9, 2, 9]], [[4, 6, 3, 6]], [1, 8, 2], 7], [[[3, 1, 5, 5], []], 9, 6], [0, 4, 1, 6, 0]]
[[], [0, 5], [[4, 8, [10], 3, [8, 9, 4, 4, 4]]]]
[[], [[0, 1, [4, 1,